<a href="https://colab.research.google.com/github/GadirajuSanjayvarma/EVA4/blob/master/S4/Eva_4_Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#importing necessary modules for training and testing model
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
#cnn model for used for training and testing of MNIST dataset
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #Convolution layers declaration of model
        self.conv1 = nn.Conv2d(1, 32, 3,bias=False) 
        self.conv2 = nn.Conv2d(32, 32, 3,bias=False)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(32, 16, 1,bias=False)
        self.conv4 = nn.Conv2d(16, 16, 3,bias=False)
        self.conv5 = nn.Conv2d(16, 16, 3,bias=False)
        self.conv6 = nn.Conv2d(16, 16, 3,bias=False)
        self.conv7 = nn.Conv2d(16, 10, 3,bias=False)
        self.conv8 = nn.Conv2d(16,10 , 3,bias=False)
        #Adaptive global average pooling layer declaration of model
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        #Batch normalisation functions declaration of model
        self.batchnorm1=nn.BatchNorm2d(32) # Batch normalisation function declaration of layers having channels=32
        self.batchnorm2=nn.BatchNorm2d(16) # Batch normalisation function declaration of layers having channels=16
        #Drop out regularization function declaration with probability 5%
        self.drop0 = nn.Dropout2d(0.05)
    def forward(self, x):
        x=self.conv1(x)
        x=F.relu(x)
        x=self.batchnorm1(x)
        x=self.drop0(x)
        x=self.conv2(x)
        x=F.relu(x)
        x=self.pool1(x)
        x=self.drop0(x)
        x=self.conv3(x)
        x=F.relu(x)
        x=self.batchnorm2(x)
        x=self.drop0(x)
        x=self.conv4(x)
        x=F.relu(x)
        x=self.batchnorm2(x)
        x=self.drop0(x)
        x=self.conv5(x)
        x=F.relu(x)
        x=self.batchnorm2(x)
        x=self.drop0(x)
        x=self.conv6(x)
        x=F.relu(x)
        x=self.conv7(x)
        x=F.relu(x)
        x=self.avgpool(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [11]:
#printing summary of model by assigning cuda or cpu based on GPU availability
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 26, 26]             288
       BatchNorm2d-2           [-1, 32, 26, 26]              64
         Dropout2d-3           [-1, 32, 26, 26]               0
            Conv2d-4           [-1, 32, 24, 24]           9,216
         MaxPool2d-5           [-1, 32, 12, 12]               0
         Dropout2d-6           [-1, 32, 12, 12]               0
            Conv2d-7           [-1, 16, 12, 12]             512
       BatchNorm2d-8           [-1, 16, 12, 12]              32
         Dropout2d-9           [-1, 16, 12, 12]               0
           Conv2d-10           [-1, 16, 10, 10]           2,304
      BatchNorm2d-11           [-1, 16, 10, 10]              32
        Dropout2d-12           [-1, 16, 10, 10]               0
           Conv2d-13             [-1, 16, 8, 8]           2,304
      BatchNorm2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:

#functions used to download mnist dataset and assigning batch size 
torch.manual_seed(1)
batch_size = 64


kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm_notebook
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm_notebook(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:51: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.




Test set: Average loss: 0.0576, Accuracy: 9817/10000 (98.17%)





Test set: Average loss: 0.0414, Accuracy: 9867/10000 (98.67%)





Test set: Average loss: 0.0300, Accuracy: 9903/10000 (99.03%)





Test set: Average loss: 0.0333, Accuracy: 9895/10000 (98.95%)



...



Test set: Average loss: 0.0243, Accuracy: 9929/10000 (99.29%)





Test set: Average loss: 0.0216, Accuracy: 9930/10000 (99.30%)





Test set: Average loss: 0.0246, Accuracy: 9925/10000 (99.25%)





Test set: Average loss: 0.0261, Accuracy: 9919/10000 (99.19%)





Test set: Average loss: 0.0229, Accuracy: 9934/10000 (99.34%)

